# Load data from Smart-Kages into `movement` datasets
Load all DLC .h5 pose files for each kage and concatenate them
into a single `movement` dataset.

Assign a datetime index across the `time` dimension for easy access.

Save the resulting dataset to a netCDF file.

In [10]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
from matplotlib import pyplot as plt
from movement.io import load_poses

from smart_kages_movement.io import load_background_frame

## Configuration
Define some global variables and paths.

In [11]:
FPS = 2  # frames per second
PIXELS_PER_CM = 10  # pixels per centimetre (need to double-check this)
TIME_PRECISION = "ns"

# Configure seaborn for prettier plots
sns.set_context("notebook")
sns.set_style("ticks")

In [12]:
data_dir = Path.home() / "Data" / "Smart-Kages"
analysis_dir = data_dir / "movement_analysis"
# csv file generated by the 01 notebook
df_path = analysis_dir / "all_segments.csv"
overlaps_path = analysis_dir / "segment_overlaps.csv"


for path in [data_dir, analysis_dir, df_path, overlaps_path]:
    if not path.exists():
        print(f"Path does not exist: {path}")

## Load CSV files as datframes

We load the dataframe containing the paths to all the DLC .h5 files, which is generated in the 01 notebook.

In [13]:
df = pd.read_csv(
    df_path,
    index_col=[0, 1, 2],
    dtype={
        "date": str,
        "hour": str,
        "n_frames": int,
    },
    parse_dates=["start_datetime", "end_datetime"],
)
df.head()

start_datetime            end_datetime  \
kage  date     hour                                               
kage1 20240403 09   2024-04-03 09:54:24 2024-04-03 09:59:56.500   
               10   2024-04-03 10:00:06 2024-04-03 10:59:57.500   
               11   2024-04-03 11:01:07 2024-04-03 11:59:59.000   
               12   2024-04-03 12:01:08 2024-04-03 12:59:57.500   
               13   2024-04-03 13:01:07 2024-04-03 13:59:56.000   

                                   duration  n_frames  n_channels  height  \
kage  date     hour                                                         
kage1 20240403 09    0 days 00:05:32.500000       665           3     376   
               10    0 days 00:59:51.500000      7183           3     376   
               11           0 days 00:58:52      7064           1     376   
               12    0 days 00:58:49.500000      7059           3     376   
               13           0 days 00:58:49      7058           3     376   

                     width                                     pose_file_path  \
kage  date     hour                                                             
kage1 20240403 09      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               10      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               11      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               12      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   
               13      500  /Users/nsirmpilatze/Data/Smart-Kages/kage1/ana...   

                                                       video_file_path  
kage  date     hour                                                     
kage1 20240403 09    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               10    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               11    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               12    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...  
               13    /Users/nsirmpilatze/Data/Smart-Kages/kage1/vid...

We see that the dataframe holds various information on each 1-hour segment of the data.

The index is hierarchical, organising the data first by `kage`, then by `date`, and finally by `hour`.

Of special relevance to us here:
- Paths: `pose_file_path`, `video_file_path`
- `start_datetime` and `end_datetime` for each segment

Now let's see which segments overlap with each other (these were pre-computed in the 01 notebook).

In [14]:
overlaps = pd.read_csv(
    overlaps_path,
    index_col=0,
    parse_dates=["end_A", "start_B"],
)
overlaps

,segment_A,segment_B,end_A,start_B,overlap_duration
0,"('kage1', '20240426', '07')","('kage1', '20240426', '08')",2024-04-26 08:05:00.500,2024-04-26 08:00:06,00:04:54.500000
1,"('kage1', '20240427', '07')","('kage1', '20240427', '08')",2024-04-27 08:06:00.000,2024-04-27 08:00:06,00:05:54
2,"('kage1', '20240428', '07')","('kage1', '20240428', '08')",2024-04-28 08:06:00.500,2024-04-28 08:00:06,00:05:54.500000
3,"('kage1', '20240429', '07')","('kage1', '20240429', '08')",2024-04-29 08:06:00.000,2024-04-29 08:00:06,00:05:54
4,"('kage1', '20240430', '07')","('kage1', '20240430', '08')",2024-04-30 08:05:00.000,2024-04-30 08:00:06,00:04:54
5,"('kage1', '20240502', '07')","('kage1', '20240502', '08')",2024-05-02 08:07:00.000,2024-05-02 08:00:06,00:06:54
6,"('kage1', '20240505', '07')","('kage1', '20240505', '08')",2024-05-05 08:09:00.500,2024-05-05 08:00:06,00:08:54.500000
7,"('kage1', '20240507', '07')","('kage1', '20240507', '08')",2024-05-07 08:08:00.000,2024-05-07 08:00:06,00:07:54
8,"('kage3', '20240425', '06')","('kage3', '20240425', '07')",2024-04-25 06:59:52.500,2024-04-25 06:59:21,00:00:31.500000
9,"('kage3', '20240504', '06')","('kage3', '20240504', '07')",2024-05-04 06:59:39.000,2024-05-04 06:59:21,00:00:18


## Load all data from a given kage
We will create a function that, given a kage name, will load all the data from the DLC .h5 files and merge them into a single `movement` dataset.

In [15]:
def kage_to_movement_ds(
    df: pd.DataFrame,
    kage: str,
    overlaps: pd.DataFrame | None = None,
) -> tuple[xr.Dataset, np.ndarray]:
    """Load all poses for a given kage and return an xarray Dataset.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the paths to pose files as well as metadata
        for each 1-hour segment.
    kage : str
        The name of the kage to process, e.g., "kage1", "kage2", etc.
    overlaps : pd.DataFrame | None
        Optional, DataFrame containing information about overlapping segments.

    Returns
    -------
    xr.Dataset
        An xarray Dataset containing the poses for the specified kage,
        with time coordinates assigned based on the corrected timestamps.
    np.ndarray
        A background image (numpy array) loaded from the middle segment
        of the kage, used for visualization purposes.

    Notes
    -----
    The returned Dataset will have two time coordinates:
    - ``time``: the primary time coordinate based on the corrected timestamps.
    - ``seconds_since_start``: the secondary time coordinate representing
       seconds elapsed since the start of the kage.
    """

    def _is_monotonic_increasing(arr):
        """Check if a 1D array is monotonically increasing."""
        return (arr[1:] >= arr[:-1]).all()

    print(f"Processing kage: {kage}")
    df_kage = df.loc[kage]
    df_kage = df_kage.sort_index()  # ensure chronological order
    n_days = df_kage.index.get_level_values("date").nunique()
    print(f"Number of days: {n_days}")
    n_segments = df_kage.shape[0]
    print(f"Number of 1-hour segments: {n_segments}")

    kage_start_datetime = pd.Timestamp(
        df_kage["start_datetime"].iloc[0], unit=TIME_PRECISION
    )

    ds_segments = []  # List of xarray Datasets for each 1-hour segment

    for date, hour in df_kage.index:
        # Load the pose data for the current 1-hour segment
        poses = load_poses.from_file(
            df_kage.loc[(date, hour), "pose_file_path"],
            source_software="DeepLabCut",
            fps=FPS,
        )

        # Assert that length of tracks matches the number of video frames
        n_frames = df_kage.loc[(date, hour), "n_frames"]
        assert poses.sizes["time"] == n_frames, (
            f"Number of tracked timepoints ({poses.sizes['time']}) does not "
            f"match the number of frames ({n_frames}) for {date} at {hour}!"
        )

        # Create timestamps starting from the segment start datetime
        # at the specified FPS
        segment_start_datetime = pd.Timestamp(
            df_kage.loc[(date, hour), "start_datetime"], unit=TIME_PRECISION
        )
        timestamps = pd.date_range(
            start=segment_start_datetime,
            periods=n_frames,
            freq=pd.Timedelta(seconds=1 / FPS),
            unit=TIME_PRECISION,
        )

        # Sanity check: final timestamp not differ by more than 1 frame
        # from the segment end datetime
        segment_end_datetime = pd.Timestamp(
            df_kage.loc[(date, hour), "end_datetime"], unit=TIME_PRECISION
        )
        tolerance = pd.Timedelta(1 / FPS, "sec")
        assert timestamps[-1] - segment_end_datetime <= tolerance, (
            f"Final timestamp {timestamps[-1]} differs by more than "
            f"{tolerance} from segment end datetime {segment_end_datetime}"
        )

        # assign time coordinates to the actual datetime timestamps
        poses = poses.assign_coords(time=timestamps)
        poses.attrs["time_unit"] = f"datetime64[{TIME_PRECISION}]"

        # If this segment overlaps with the next one,
        # we'll delete overlapping frames at the end of this segment
        segment_str = f"('{kage}', '{date}', '{hour}')"
        if (overlaps is not None) and segment_str in overlaps[
            "segment_A"
        ].values:
            row_index = overlaps.index[overlaps["segment_A"] == segment_str][0]
            # Delete everything that comes after the start time of segment B
            next_segment_start = pd.Timestamp(
                overlaps.loc[row_index, "start_B"], unit=TIME_PRECISION
            )
            poses = poses.sel(time=slice(None, next_segment_start - tolerance))
            n_removed = n_frames - poses.sizes["time"]
            print(
                f"Removed {n_removed} overlapping frames at the end of "
                f"segment {date} {hour} for {kage}."
            )

        # add to list of loaded segments
        ds_segments.append(poses)

    # Combine all segments into a single xarray Dataset
    ds_kage = xr.concat(ds_segments, dim="time")
    ds_kage.attrs["kage"] = kage
    ds_kage.attrs["kage_start_datetime"] = kage_start_datetime.isoformat()

    # Ensure the concatenated timestamps are monotonic increasing
    assert _is_monotonic_increasing(ds_kage.time.values), (
        f"Combined timestamps for {kage} are not monotonic increasing!"
    )

    # Assign secondary time coordinate as seconds elapsed since kage start
    seconds_since_kage_start = (
        ds_kage.time.data - np.datetime64(kage_start_datetime)
    ) / pd.Timedelta("1s")
    ds_kage = ds_kage.assign_coords(
        seconds_elapsed=("time", seconds_since_kage_start)
    )

    # load image to use as background frame
    video_path = df_kage.iloc[n_segments // 2]["video_file_path"]
    background_img = load_background_frame(
        video_path=video_path, i=0, n_average=100
    )
    print(f"Loaded background image for {kage} from {video_path} \n")

    return ds_kage, background_img

Now let's create a combined `movement` dataset for each kage.

We also assign a background image to each kage, which is used for visualisation purposes.

In [16]:
ds_dict = {}  # List of `movement` datasets for each kage
img_dict = {}  # List of background images for each kage

for kage in df.index.get_level_values("kage").unique():
    ds_dict[kage], img_dict[kage] = kage_to_movement_ds(df, kage, overlaps)

Processing kage: kage1
Number of days: 35
Number of 1-hour segments: 760
Removed 589 overlapping frames at the end of segment 20240426 07 for kage1.
Removed 708 overlapping frames at the end of segment 20240427 07 for kage1.
Removed 709 overlapping frames at the end of segment 20240428 07 for kage1.
Removed 708 overlapping frames at the end of segment 20240429 07 for kage1.
Removed 588 overlapping frames at the end of segment 20240430 07 for kage1.
Removed 828 overlapping frames at the end of segment 20240502 07 for kage1.
Removed 1069 overlapping frames at the end of segment 20240505 07 for kage1.
Removed 948 overlapping frames at the end of segment 20240507 07 for kage1.
Loaded background image for kage1 from /Users/nsirmpilatze/Data/Smart-Kages/kage1/videos/2024/04/23/kage1_20240423_130002.mp4 

Processing kage: kage3
Number of days: 39
Number of 1-hour segments: 855
Removed 63 overlapping frames at the end of segment 20240425 06 for kage3.
Removed 36 overlapping frames at the end o

In [17]:
# Inspect the dataset for kage1
ds_dict["kage1"]

<xarray.Dataset> Size: 1GB
Dimensions:          (time: 5431121, space: 2, keypoints: 8, individuals: 1)
Coordinates:
  * space            (space) <U1 8B 'x' 'y'
  * keypoints        (keypoints) <U10 320B 'snout' 'leftear' ... 'tailbase'
  * individuals      (individuals) <U12 48B 'individual_0'
  * time             (time) datetime64[ns] 43MB 2024-04-03T09:54:24 ... 2024-...
    seconds_elapsed  (time) float64 43MB 0.0 0.5 1.0 ... 3.194e+06 3.194e+06
Data variables:
    position         (time, space, keypoints, individuals) float64 695MB 309....
    confidence       (time, keypoints, individuals) float64 348MB 0.9382 ... ...
Attributes:
    source_software:      DeepLabCut
    ds_type:              poses
    fps:                  2.0
    time_unit:            datetime64[ns]
    source_file:          /Users/nsirmpilatze/Data/Smart-Kages/kage1/analysis...
    kage:                 kage1
    kage_start_datetime:  2024-04-03T09:54:24

## Save the combined datasets to netCDF files
Finally, we save the combined `movement` datasets for each kage to a netCDF file.

We also save the background image for each kage in the same directory.

In [19]:
for kage, ds in ds_dict.items():
    print(f"Saving dataset for {kage}...")
    kage_dir = analysis_dir / kage
    kage_dir.mkdir(parents=True, exist_ok=True)
    ds_file_path = kage_dir / f"{kage}.nc"
    ds.to_netcdf(ds_file_path, mode="w", engine="netcdf4", format="NETCDF4")
    print(f"Dataset for {kage} saved to {ds_file_path}.")

    img_file_path = kage_dir / f"{kage}_background.png"
    plt.imsave(img_file_path, img_dict[kage])
    print(f"Background image for {kage} saved to {img_file_path}.\n")

Saving dataset for kage1...
Dataset for kage1 saved to /Users/nsirmpilatze/Data/Smart-Kages/movement_analysis/kage1/kage1.nc.
Background image for kage1 saved to /Users/nsirmpilatze/Data/Smart-Kages/movement_analysis/kage1/kage1_background.png.

Saving dataset for kage3...
Dataset for kage3 saved to /Users/nsirmpilatze/Data/Smart-Kages/movement_analysis/kage3/kage3.nc.
Background image for kage3 saved to /Users/nsirmpilatze/Data/Smart-Kages/movement_analysis/kage3/kage3_background.png.

